In [1]:
"""
MongoDB Database Connection and Operations Module

This module provides functions to connect to a MongoDB database
and perform various data extraction operations.
"""

import pymongo
from pymongo import MongoClient
import pandas as pd
import json
from datetime import datetime
import os
from urllib.parse import quote_plus

# Database connection constants
USER_NAME = "alfred"
PASS = "alfred-coco-cola"
MONGO_CLIENT = 'alfred-coco-cola'
MONGO_COLLECTION = 'email_threads'
MONGO_HOST = '172.178.91.142'
MONGO_PORT = 27017



/var/folders/4s/gb51cxx975xgdgnzpf1r7mr40000gp/T/ipykernel_2466/2904192211.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
import pymongo
import pandas as pd
from pymongo import MongoClient
import json
from bson import json_util

# Connect to the MongoDB database
# Assuming the connection details are the same as in the app.py file
client = MongoClient('localhost', 27017)  # Default MongoDB connection
db = client['alfred-coco-cola']  # Use the database name from the codebase
collection = db['email_threads']  # Access the email_threads collection

# Fetch all documents from the collection
email_threads = list(collection.find())

# Print the count of documents
print(f"Total documents in email_threads collection: {len(email_threads)}")

# Convert to a more readable format
# This handles MongoDB-specific types like ObjectId
def parse_json(data):
    return json.loads(json_util.dumps(data))

# Display the first document to see its structure
if email_threads:
    print("\nSample document structure:")
    print(json.dumps(parse_json(email_threads[0]), indent=2))

# Convert the data to a pandas DataFrame for easier analysis
# First flatten the documents to handle nested structures
flat_data = [parse_json(doc) for doc in email_threads]
df = pd.json_normalize(flat_data)

# Display the DataFrame
print("\nDataFrame view of email_threads:")
display(df)

# If you want to see all documents in detail
print("\nAll documents in email_threads:")
for i, thread in enumerate(email_threads):
    print(f"\n--- Document {i+1} ---")
    print(json.dumps(parse_json(thread), indent=2))

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 680774f04fbaa6c633a27322, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>